In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [ ]:
import sys
sys.version

# Introduction

In [ ]:
def random_walk_faster(n=1000):
    from itertools import accumulate
    steps = random.sample((-1,+1)*n, k=n)
    return [0]+list(accumulate(steps))

#walk = random_walk_faster(1000)

In [ ]:
import random
random.choices([-1,+1], k=20)

In [ ]:
import random
from itertools import accumulate
steps = random.sample((-1,+1)*10, k=10)
print([0]+steps)
[0]+list(accumulate(steps))

In [ ]:
timeit("random_walk_faster(n=10000)", globals())

In [ ]:
def random_walk_fastest(n=1000):
    steps = 2*np.random.randint(0, 2, size=n) - 1
    return np.cumsum(steps)

walk = random_walk_fastest(1000)

In [ ]:
timeit("random_walk_fastest(n=10000)", globals())

# Anatomy of an array

In [ ]:
Z = np.ones(4*1000000, np.float32)

In [ ]:
timeit("Z[...] = 0", globals())

In [ ]:
timeit("Z.view(np.int8)[...] = 0", globals())

In [ ]:
Z = np.arange(9).reshape(3,3).astype(np.int16)

In [ ]:
print(Z.itemsize) # 2 bytes

In [ ]:
print(Z.shape)

In [ ]:
print(Z.ndim)

In [ ]:
Z.shape[1]

In [ ]:
Z.size

## Views and copies

In [ ]:
Z = np.zeros(9)

In [ ]:
Z

In [ ]:
# indexing returns a view
Z_view = Z[2:4]

In [ ]:
Z_view

In [ ]:
Z_view[...] = 1

In [ ]:
Z_view

In [ ]:
Z

In [ ]:
Z = np.zeros(9)

In [ ]:
Z

In [ ]:
# fancy indexing returns a copy
Z_copy = Z[[0,1,2]]

In [ ]:
Z_copy

In [ ]:
Z_copy[...] = 1

In [ ]:
Z_copy

In [ ]:
Z

In [ ]:
Z = np.zeros(9)

In [ ]:
index = [0,1,2]

In [ ]:
Z[index] = 1

In [ ]:
Z

If you are unsure if the result of you indexing is a view or a copy, you can check what is the base of your result. If it is None, then you result is a copy:

In [ ]:
Z = np.random.uniform(0,1,(5,5))

In [ ]:
Z

In [ ]:
Z1 = Z[:3,:]
Z2 = Z[[0,1,2], :]

In [ ]:
Z1

In [ ]:
Z2

In [ ]:
print(Z1.base is Z) # Is it a view?

In [ ]:
print(Z2.base is None) # Is it a copy?

Note that some numpy functions return a view when possible (e.g. ravel) while some others always return a copy (e.g. flatten):

In [ ]:
Z = np.zeros((5,5))

In [ ]:
Z

In [ ]:
Z.ravel()

In [ ]:
Z.ravel().base is Z # tries to return a view

In [ ]:
Z.flatten()

In [ ]:
Z.flatten().base is Z # always return a copy

Temporary copy workaround

In [ ]:
X = np.ones(10, dtype=np.int)
Y = np.ones(10, dtype=np.int)
np.multiply(X, 2, out=X)
np.multiply(Y, 2, out=Y)
np.add(X, Y, out=X)

In [ ]:
np.arange(9)

In [ ]:
Z1 = np.arange(10)

In [ ]:
Z1

In [ ]:
Z2 = Z1[-1::-1] # Z1[start:stop:step]
Z2

In [ ]:
print(np.allclose(Z1[0::1], Z2))

# Code vectorization

In [ ]:
# Python to add two lists
def add_python(Z1,Z2):
    return [z1+z2 for (z1,z2) in zip(Z1,Z2)]

In [ ]:
# Code vectorized using numpy
def add_numpy(Z1,Z2):
    return np.add(Z1,Z2)

## The Game of Life

In [ ]:
# initial board is a list of lists
Z = [[0,0,0,0,0,0],
     [0,0,0,1,0,0],
     [0,1,0,1,0,0],
     [0,0,1,1,0,0],
     [0,0,0,0,0,0],
     [0,0,0,0,0,0]]

In [ ]:
def compute_neighbours(Z):
    shape = len(Z), len(Z[0])
    N  = [[0,]*(shape[0]) for i in range(shape[1])]
    for x in range(1,shape[0]-1):
        for y in range(1,shape[1]-1):
            N[x][y] = Z[x-1][y-1]+Z[x][y-1]+Z[x+1][y-1] \
                    + Z[x-1][y]            +Z[x+1][y]   \
                    + Z[x-1][y+1]+Z[x][y+1]+Z[x+1][y+1]
    return N

In [ ]:
def iterate(Z):
    N = compute_neighbours(Z)
    for x in range(1,shape[0]-1):
        for y in range(1,shape[1]-1):
             if Z[x][y] == 1 and (N[x][y] < 2 or N[x][y] > 3):
                 Z[x][y] = 0
             elif Z[x][y] == 0 and N[x][y] == 3:
                 Z[x][y] = 1
    return Z